# Circuit learning module: Pennylane with SPSA

This code is inspired by [Quanthoven](https://github.com/CQCL/Quanthoven/blob/main/experiment.ipynb) and [Pennylane demo](https://pennylane.ai/qml/demos/spsa.html).

In [1]:
import json
import os
import sys
import glob
import warnings
import collections
from pathlib import Path
import numpy as np
import pickle
import pennylane as qml
from sympy import default_sort_key
import torch
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from inspect import signature
from noisyopt import minimizeSPSA
from utils import transform_into_pennylane_circuits, read_diagrams, get_symbols, create_labeled_classes, multi_class_loss

this_folder = os.path.abspath(os.getcwd())
nshot = 1000

In [2]:
def genbin(n, bs=''):
    if len(bs) == n:
        return bs
    else:
        return np.array([genbin(n, bs + '0'), genbin(n, bs + '1')]).flatten()

## Read circuit data

We read the circuits from the pickled files.

In [3]:
# Select workload
#workload = "execution_time"
workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//"\
            + workload + "//" + workload_size + "//circuits//"\
            + str(classification) + "//" + str(layers) + "_layer//"\
           + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count)\
            + "_n_wire_count//"

In [4]:
training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [5]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

In [6]:
n_qubits = 15
dev = qml.device("lightning.qubit", wires=n_qubits, shots=nshot)

qml_training_circuits, train_symbols = transform_into_pennylane_circuits(training_circuits, n_qubits, dev)
qml_test_circuits, test_symbols = transform_into_pennylane_circuits(test_circuits, n_qubits, dev)
qml_validation_circuits, val_symbols = transform_into_pennylane_circuits(validation_circuits, n_qubits, dev)

In [7]:
#training_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_training_circuits.values()])
#validation_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_validation_circuits.values()])
#test_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_test_circuits.values()])

In [8]:
print("Number of training circuits: ", len(qml_training_circuits))
print("Number of validation circuits: ", len(qml_validation_circuits))
print("Number of test circuits: ", len(qml_test_circuits))

Number of training circuits:  448
Number of validation circuits:  113
Number of test circuits:  112


## Read training and test data

In [9]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Post-selection

In [10]:
def post_select_circuit_samples(circuit_samples, n_qubits, post_selection):
    selected_samples = []
    post_select_array = np.array([0]*(n_qubits - post_selection))
    for circuit_sample in circuit_samples:
        if np.array_equal(circuit_sample[post_selection - 1 :-1], post_select_array):
            res = circuit_sample[0:post_selection]
            selected_samples.append(res)
            #if circuit_sample[0] == 1:
            #    selected_samples.append(1)
            #else:
            #    selected_samples.append(0)
    return selected_samples

## Cost function

In [11]:
def make_pred_fn(circuits):
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

In [12]:
def acc_from_dict(dict_predictions, dict_labels):
    total_acc = 0
    for query_id in dict_predictions:
        y_meas = np.array(dict_predictions[query_id]).flatten()
        max_index = np.argmax(y_meas)
        total_acc += int(int(dict_labels[query_id][max_index]) == 1)
    return total_acc / len(dict_predictions)


def loss_from_dict(dict_predictions, dict_labels):
    total_loss = 0
    for query_id in dict_predictions:
        x = np.array(dict_labels[query_id])
        y_pred = np.array(dict_predictions[query_id]).flatten()
        #print(x, y_pred)
        total_loss += -np.sum(x * np.log(y_pred)) / len(x)
        #print(total_loss)
    return total_loss

In [13]:
def make_cost_fn(qnodes, labels):    
    def cost_spsa(params, **kwargs):
        predictions = {}
        for i, q in enumerate(qnodes):
            circuit = qnodes[q]["qml_circuit"]
            measurement = circuit(params)
            post_selected_samples = post_select_circuit_samples(measurement, n_qubits, 2)
            post_selected_samples = [tuple(map(int, t)) for t in post_selected_samples]
            #print(post_selected_samples)
            counts = collections.Counter(post_selected_samples)
            res = []
            for s in genbin(classification):
                t = tuple(map(int, s))
                #print(t, counts)
                if t in counts:
                    #print(counts[t]/len(post_selected_samples))
                    res.append(counts[t]/len(post_selected_samples))
                else:
                    res.append(1e-9)
            predictions[q] = res
            #if i == 4:
            #    break
        #print(predictions)
        cost = loss_from_dict(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        #print(cost)
        costs.append(cost)

        accuracy = acc_from_dict(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        #print(accuracy)
        accuracies.append(accuracy)
        
        return cost
    
    costs, accuracies = [], []
    return cost_spsa, costs, accuracies

## SPSA minimization

In [14]:
import numpy

BATCH_SIZE = 32
EPOCHS = 100
SEED = 0
result_file = workload + "_" + workload_size + "_pennylane_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

rng = np.random.default_rng(SEED)
init_params_spsa = np.array(rng.random(len(train_symbols)))
np.random.seed(SEED)

train_cost_fn, train_costs, train_accs = make_cost_fn(qml_training_circuits, training_data_labels)
dev_cost_fn, dev_costs, dev_accs = make_cost_fn(qml_validation_circuits, validation_data_labels)

def callback_fn(xk):
    valid_loss = dev_cost_fn(xk)
    train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
    train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
    valid_acc = numpy.around(float(dev_accs[-1]), 4)
    iters = int(len(train_accs)/2)
    #if iters % 200 == 0:
    info = f"Epoch: {iters}   "\
    + f"train/loss: {train_loss}   "\
    + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
    + f"train/acc: {train_acc}   "\
    + f"valid/acc: {valid_acc}"

    with open("results//" + result_file + ".txt", "a") as f:
        f.write(info + "\n")

    print(info, file=sys.stderr)
    return valid_loss

In [ ]:
result = minimizeSPSA(train_cost_fn, 
                      x0 = init_params_spsa.copy(), 
                      a = 0.0053, 
                      c = 0.00185, 
                      niter=EPOCHS, 
                      paired=False, 
                      callback=callback_fn)